# **ABAQUS INPUT FILE GENERATOR**
---
This notebook will be used to check the results of the optimization process.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# Modify to match required case
load_case = 'torsion'
stacking_sequence = 'symmetric_balanced'
data_set = '8x'
fiber_path = 'harmlin'

# Results folder
input_folder = './' + load_case + '/' + stacking_sequence + '/' + data_set + '/' + fiber_path + '/'

# Optimization results
results = pd.read_csv(input_folder + 'results.csv', sep=",")
results

,Buckling,Amplitude1,PhaseShift1,Omega1,Beta1,Amplitude2,PhaseShift2,Omega2,Beta2
0,47623.1580,-82.0448,-34.5756,0.0,90.0000,200.0000,90.0,2.0,-90.0000
1,47366.4543,200.0000,-90.0000,0.0,71.4717,200.0000,90.0,2.0,-90.0000
2,47057.6757,200.0000,90.0000,0.0,90.0000,15.7313,-90.0,2.0,5.5403
3,47551.6897,-87.0601,-31.9412,0.0,90.0000,200.0000,90.0,2.0,-90.0000
4,47042.5380,200.0000,90.0000,0.0,90.0000,18.2856,-90.0,2.0,6.0871
5,47557.5643,-87.8164,-32.2232,0.0,90.0000,200.0000,90.0,2.0,-90.0000
6,47622.1405,-80.2295,-34.4328,0.0,90.0000,200.0000,90.0,2.0,-90.0000
7,47055.1277,200.0000,90.0000,0.0,90.0000,15.9944,-90.0,2.0,6.6708
8,47061.4004,200.0000,90.0000,0.0,90.0000,15.1108,-90.0,2.0,5.3462
9,47263.4986,-200.0000,-22.2087,0.0,90.0000,200.0000,90.0,2.0,-90.0000


In [3]:
h = 705
r = 300
mesh_x = 10
x_elem = int(h / mesh_x)
plies = 8
eff_plies = 2
symmetric = True
balanced = True
t = 0.181

p = 2 * np.pi * r
mesh_yz = p / round(p / mesh_x)
n_x = int(h / mesh_x + 1)
n_yz = int(p / mesh_yz)
tot_nodes = n_x * n_yz
plot = False

Some info related to the type of analysis

In [4]:
flag_torsion = True
flag_axial = False

## **Local angles definition**
---

In [5]:
def harmlin(height, mesh_size, amplitude, phase_shift, omega, beta):
    x_elems = int(h / mesh_size)
    x = np.linspace(mesh_size / 2, h - mesh_size / 2, x_elems)  # normalized domain
    y = amplitude * (np.sin(omega * (2 * np.pi / h) * x + phase_shift)) + x * np.tan(beta)
    simb_dy = 2 * np.pi / h * omega * amplitude * np.cos(omega * (2 * np.pi / h) * x + phase_shift) + np.tan(beta)
    return np.arctan(simb_dy) * 180 / np.pi

In [6]:
row = 2
ply_1 = results.loc[row, ['Amplitude1', 'PhaseShift1', 'Omega1', 'Beta1']].to_numpy()
ply_2 = results.loc[row, ['Amplitude2', 'PhaseShift2', 'Omega2', 'Beta2']].to_numpy()
arr_plies = np.array([ply_1, ply_2])

In [7]:
col = ['Theta' + str(i) for i in range(1, x_elem + 1)]
angles = {}
for ply in range(0, eff_plies):
    ang = np.empty((0, x_elem), int)
    ang = np.array(harmlin(h, mesh_x, arr_plies[ply][0], arr_plies[ply][1] * np.pi / 180, arr_plies[ply][2], arr_plies[ply][3] * np.pi / 180))
    print(ang)
    angles.update({'Ply' + str(ply+1): ang})  

[90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90.
 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90.
 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90.
 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90. 90.]
[  6.9532546    9.72765148  12.32599203  14.66460273  16.6763762
  18.31109235  19.53371538  20.3216924   20.66222169  20.5502021
  19.98728267  18.9821794   17.55220512  15.72572716  13.54499121
  11.06846063   8.37163684   5.54543948   2.69176673  -0.08325617
  -2.67697172  -4.99709357  -6.96643448  -8.52493675  -9.62938835
 -10.25177381 -10.37733007 -10.00317179  -9.13798997  -7.80292623
  -6.03331859  -3.880625    -1.41352824   1.2828392    4.11027349
   6.9634372    9.73736828  12.33491513  14.6724601   16.68295745
  18.31624569  19.53733905  20.32372437  20.66263075  20.54898226
  19.98445207  18.97778365  17.54632479  15.71848766  13.53657273
  11.05910435   8.36164401   5.53516044   2.68158018  -0.

Define nodes coordinate:

In [8]:
x =  np.linspace(0, h, n_x)
theta = np.arange(0, 2 * np.pi, 2 * np.pi / n_yz)
#theta = np.linspace(0, 2 * np.pi, n_yz) #last node coincide with first
y = r * np.cos(theta)
z = r * np.sin(theta)

coord_x = np.zeros(n_yz)
coord_y = y
coord_z = z
for v_elem in range(1, n_x):
    coord_x = np.append(coord_x, np.ones(n_yz) * x[v_elem])
    coord_y = np.append(coord_y, y)
    coord_z = np.append(coord_z, z)

In [9]:
nodes = {}
for n in range(0, tot_nodes):
    name = 'Node' + str(n + 1)
    nodes[name] = (round(coord_x[n], 6), round(coord_y[n], 6), round(coord_z[n], 6))

Nodes for constraint and load and sets

In [10]:
ref_nodes = {'Load': (h, 0, 0), 'Constraint': (0, 0, 0)} 

set_nodes ={'LoadNodes': [i for i in range(n_x * n_yz - n_yz + 1, n_x * n_yz + 1)],
            'ConstraintNodes': [i for i in range(1, n_yz + 1)]}

In [11]:
elements = {}
foo = 1
for n_v in range(0, n_x - 1):
    for n_h in range(0, n_yz - 1):
        name = 'Element' + str(foo)
        foo += 1
        n1 = n_v * n_yz + n_h + 1
        n2 =  n1 + 1
        n3 = n2 + n_yz
        n4 = n1 + n_yz
        elements[name] = (n1, n2, n3, n4)
    mid1 = n2 - n_h - 1
    mid2 = n3 - n_h - 1
    elements['Element' + str(foo)] = (n2, mid1, mid2, n3)
    foo += 1
tot_elems = foo - 1

In [12]:
if flag_axial:
    heading = 'Analysis of a variable stiffness cylindrical shell in axial compression'
elif flag_torsion:
    heading = 'Analysis of a variable stiffness cylindrical shell in torsion'
job = 'Buckling analysis'
model = 'Variable stiffness cylindrical shell'

In [13]:
directory = input_folder + 'abaqus/'

try:
    if not os.path.isdir(directory):
        os.makedirs(directory)
except OSError:
    print('Error: Creating directory. ' + directory)

In [14]:
f = open(directory + "optimum_config.inp","w+")
f.write('*HEADING\n' + heading + '\n')
f.write('** Job name: ' + job + '\n')
f.write('** Model name: ' + model + '\n')
f.write('** Generated by: S.F. Pitton\n')

29

In [15]:
f.write('**\n** PARTS\n**\n')
part_name = 'Cylinder'
f.write('*Part, name = ' + part_name + '\n')
f.write('*End Part\n')

10

In [16]:
f.write('**\n** ASSEMBLY\n**\n')
assembly_name = 'Assembly_Cylinder'
f.write('*Assembly, name = ' + assembly_name + '\n**\n')
instance_name = 'Instance_Cylinder'
f.write('*Instance, name = ' + instance_name + ', part = ' + part_name + '\n')

53

In [17]:
f.write('**\n** MESH\n**\n')

14

In [18]:
f.write('*Node\n')
#f.write('*Node, nset = all\n')
for ID_node in range(1, tot_nodes + 1):
    nx = nodes['Node' + str(ID_node)][0]
    ny = nodes['Node' + str(ID_node)][1]
    nz = nodes['Node' + str(ID_node)][2]
    f.write('%4.0f, %12.6f, %12.6f, %12.6f\n' % (ID_node, nx, ny, nz))

In [19]:
f.write('**\n** ELEMENTS\n**\n')
f.write('*Element, type=S4R\n')
#f.write('*Element, type=S4R, elset = all\n')
for ID_elem in range(1, tot_elems + 1):
    n1 = elements['Element' + str(ID_elem)][0]
    n2 = elements['Element' + str(ID_elem)][1]
    n3 = elements['Element' + str(ID_elem)][2]
    n4 = elements['Element' + str(ID_elem)][3]
    f.write('%4.0f, %4.0f, %4.0f, %4.0f, %4.0f\n' % (ID_elem, n1, n2, n3, n4))

In [20]:
f.write('**\n** SETS OF ELEMENTS\n**\n')
for e_set in range(0, n_x - 1):
    f.write('*Elset, elset = set' + str(e_set + 1) + ', GENERATE\n')
    e_start = 1 + e_set * n_yz
    e_end = n_yz + e_set * n_yz
    inc = 1
    f.write('%4.0f, %4.0f, %4.0f\n' % (e_start, e_end, inc))

In [21]:
f.write('**\n** SECTION\n**\n')
for j in range(1, n_x):
    if symmetric:
        f.write('*Shell Section, elset = set' + str(j) + ', composite, layup = vs' + str(j) + ', symmetric\n')
    else:
        f.write('*Shell Section, elset = set' + str(j) + ', composite, layup = vs' + str(j) + '\n')

    for i in range(0, eff_plies):
        f.write('%1.4f, %d, AS4/8552, %4.1f, ply%d \n' % (t, 3, angles['Ply' + str(i + 1)][j - 1], i + 1))

        if balanced:
                f.write('%1.4f, %d, AS4/8552, %4.1f, ply%d \n' %
                        (t, 3, -angles['Ply' + str(i + 1)][j - 1], i + eff_plies + 1))

In [22]:
f.write('**\n*End Instance\n**\n')

20

In [23]:
f.write('**\n*Node\n')
foo = 1 
for key in ref_nodes.keys():
    nx = ref_nodes[key][0]
    ny = ref_nodes[key][1]
    nz = ref_nodes[key][2]
    f.write('%4.0f, %12.6f, %12.6f, %12.6f\n' % (tot_nodes + foo, nx, ny, nz))
    foo += 1

Create set of nodes

In [24]:
f.write('**\n** SET OF NODES\n**\n')
for key in set_nodes.keys():
    f.write('*Nset, nset = "' + key + '", GENERATE, instance =' + instance_name + '\n')
    n_start = set_nodes[key][0]
    n_end = set_nodes[key][-1]
    inc = 1
    f.write('%4.0f, %4.0f, %4.0f\n' % (n_start, n_end, inc))

Crate tied constraint

In [25]:
f.write('**\n** CONSTRAINTS\n**\n')
foo = 1
for key in set_nodes.keys():
    f.write('*Rigid Body, ref node = %4.0f,' % (tot_nodes + foo))
    f.write(' tie nset = "' + key + '"\n')
    foo += 1

In [26]:
f.write('**\n*End Assembly\n')

17

Crate material

In [27]:
f.write('**\n** MATERIALS\n**\n')
f.write('*Material, name = AS4/8552\n')
f.write('*Elastic, type = LAMINA\n')
f.write('%4.1f, %4.1f, %4.1f, %4.1f, %4.1f, %4.1f\n' % (141000., 10300.,   0.3, 4500., 4500., 3700.))

48

create the step

In [28]:
f.write('**\n** ----------------------------------------------------------------\n')
f.write('**\n** STEP\n**\n')
f.write('*Step, name = "Buckling force step", nlgeom = NO, perturbation\n')
f.write('Eigenvalues calculation \n')
f.write('*Buckle\n')
f.write('%d, , 600,\n' % 5)

10

Boundary conditions and loads

In [29]:
if flag_axial:
    f.write('**\n** BOUNDARY CONDITIONS\n**\n')
    f.write('** Name:Only x displacements free\n')
    f.write('*Boundary, op = NEW, load case = 1 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 1, 2, 6))
    f.write('*Boundary, op = NEW, load case = 2 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 1, 2, 6))
    f.write('** Name: Clamped edge\n')
    f.write('*Boundary, op = NEW, load case = 1 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 2, 1, 6))
    f.write('*Boundary, op = NEW, load case = 2 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 2, 1, 6))

    f.write('**\n** LOADS\n**\n')
    f.write('** Name: Concentrated force \n')
    f.write('*Cload \n')
    f.write('%d, %d, %2.2f\n' % (tot_nodes + 1, 1, -1.))

if flag_torsion:
    f.write('**\n** BOUNDARY CONDITIONS\n**\n')
    f.write('** Name: Constraint y and z displacements\n')
    f.write('*Boundary, op = NEW, load case = 1 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 1, 2, 3))
    f.write('*Boundary, op = NEW, load case = 2 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 1, 2, 3))
    f.write('** Name: Constraint rotations around y and z\n')
    f.write('*Boundary, op = NEW, load case = 1 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 1, 5, 6))
    f.write('*Boundary, op = NEW, load case = 2 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 1, 5, 6))
    f.write('** Name: Clamped edge\n')
    f.write('*Boundary, op = NEW, load case = 1 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 2, 1, 6))
    f.write('*Boundary, op = NEW, load case = 2 \n')
    f.write('%d, %d, %d\n' % (tot_nodes + 2, 1, 6))

    f.write('**\n** LOADS\n**\n')
    f.write('** Name: Concentrated torque\n')
    f.write('*Cload \n')
    f.write('%d, %d, %2.2f\n' % (tot_nodes + 1, 4, -1.))

Outputs

In [30]:
f.write('**\n** OUTPUT REQUESTS\n**\n')
f.write('*Restart, write, frequency = 0\n')
f.write('**\n** FIELD OUTPUT\n**\n')
f.write('*Output, field\n')
f.write('*Node Output\n')
f.write('RF, U\n')
f.write('*Element Output, directions = YES\n')
f.write('E, S\n')
f.write('**\n** FIELD OUTPUT\n**\n')
f.write('*Output, field, variable = PRESELECT\n')
f.write('*End Step')

9

In [31]:
f.close()

In [32]:
if plot:
    %matplotlib notebook
    fig = plt.figure(figsize=(8,8))
    ax = Axes3D(fig)
    ax.scatter(coord_y, coord_z, coord_x, marker='o', linewidth=3, color='red')
    nomi = [str(i) for i in range(1, tot_nodes + 1)]
    for i in range(0, tot_nodes):
        ax.text(coord_y[i], coord_z[i],coord_x[i], nomi[i], color='black')

    ax.scatter(ref_nodes['Load'][1], ref_nodes['Load'][2], ref_nodes['Load'][0], marker='x',s=100, color='blue')
    ax.scatter(ref_nodes['Constraint'][1], ref_nodes['Constraint'][2], ref_nodes['Constraint'][0], marker='x',s=100, color='blue')
    ax.scatter(ref_nodes['Constraint'][1], ref_nodes['Constraint'][2], ref_nodes['Constraint'][0], marker='*',s=100, color='red')
    ax.quiver(0, 0, h + 0.2 * h, 0, 0, -0.2 * h, color='red', linewidth=3)

    foo_theta = np.linspace(0, 2 * np.pi, n_yz+1) 

    yy = np.outer(r * np.cos(foo_theta), np.ones(len(x)))
    zz = np.outer(r * np.sin(foo_theta),np.ones(len(x))) 
    xx = np.outer(np.ones(len(foo_theta)),x)

    # Plot the surface
    ax.plot_surface(yy, zz, xx, alpha=0.3)

    plt.show()